In [ ]:
# filename: main.py
from fastapi import FastAPI
from pydantic import BaseModel
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

app = FastAPI()

# ---- Load or define your model here ----
# For demonstration, let's assume you have already saved your trained model to 'lstm_model.h5'
model = load_model("lstm_model.h5")

# We also need to load scalers or define them exactly as in your training script
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

# Pre-fit your scalers or store them if you saved them previously
# (If you have the scaler configurations saved, load them here. Otherwise,
#  you can fit them on a known training dataset. This example is simplified.)

# ---- Request and Response Models ----
class StockPredictionRequest(BaseModel):
    ticker: str
    lookback_days: int = 5

class StockPredictionResponse(BaseModel):
    predicted_price: float
    action: str

@app.get("/")
def root():
    return {"message": "Hello from your Stock Prediction API!"}

@app.post("/predict", response_model=StockPredictionResponse)
def predict_stock(request: StockPredictionRequest):
    """
    Given a stock ticker and a lookback window, fetch data,
    run the model prediction, and return a predicted price + recommended action.
    """

    # 1) Fetch data from yfinance
    data = yf.Ticker(request.ticker).history(period="1y")
    data = data.dropna()

    # 2) Prepare your features exactly like in your script
    #    (Below is a short example; adapt to your real feature engineering pipeline)
    data["Return"] = data["Close"].pct_change()
    data["Moving_Avg_5"] = data["Close"].rolling(window=5).mean()
    data["Moving_Avg_20"] = data["Close"].rolling(window=20).mean()
    data["Volatility"] = data["Close"].rolling(window=5).std()

    data = data.dropna()

    features = ["Close", "Return", "Moving_Avg_5", "Moving_Avg_20", "Volatility"]
    X = data[features].values

    # 3) Scale X using the same scaler you trained
    #    (In a real scenario, ensure your scaler is loaded pre-fit from training data)
    X_scaled = scaler_X.fit_transform(X)

    # 4) Build the lookback sequence
    #    We'll take the last `lookback_days` days from the data to predict the next day
    if len(X_scaled) < request.lookback_days:
        # Not enough data
        return StockPredictionResponse(predicted_price=-1.0, action="Not enough data")

    last_seq = X_scaled[-request.lookback_days:]
    last_seq = np.expand_dims(last_seq, axis=0)  # shape: (1, lookback_days, num_features)

    # 5) Make prediction
    predicted_scaled = model.predict(last_seq)
    predicted_price = scaler_y.inverse_transform(predicted_scaled)[0][0]

    # 6) Simple logic for action: we can just hold, or do a naive check
    action = "Hold"
    # If you want a more sophisticated approach, you could fetch the
    # current close price and compare:
    current_close = data["Close"].iloc[-1]
    if predicted_price > current_close:
        action = "Buy"
    elif predicted_price < current_close:
        action = "Sell"

    # 7) Return the JSON response
    return StockPredictionResponse(
        predicted_price=float(predicted_price),
        action=action
    )
